In [2]:
# Dashboard code
import os
import warnings
import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz 
import hvplot.pandas
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

load_dotenv()

True

In [3]:
# Set Alpaca API key and secret
# YOUR CODE HERE!
alpaca_api_key = os.getenv("APCA_API_KEY_ID")
alpaca_secret_key = os.getenv("APCA_API_SECRET_KEY")
# Create the Alpaca API object
# YOUR CODE HERE!
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [21]:
# Define the timezone: America/New_York
tz = pytz.timezone('America/New_York')

# Get the current datetime in the specified timezone
now = datetime.now(tz)

correct_date = now #- timedelta(days=1)

# Convert to pandas Timestamp with timezone and format as ISO 8601 string
# Create a Timestamp object from the correct_date
correct_timestamp = pd.Timestamp(correct_date)

# Normalize the timestamp to midnight (00:00:00)
correct_timestamp_normalized = correct_timestamp.normalize()

# Convert the localized Timestamp object to the desired format
correct_date_str = correct_timestamp_normalized.isoformat()

# Set the tickers
tickers = ["SPY", "QQQ", "IWM", "AAPL", "MSFT", "AMZN", "GOOGL", "META", "TSLA", "NVDA", "JNJ", "JPM"]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for SPY and AGG
# YOUR CODE HERE!
current_portfolio_df = api.get_bars(tickers, timeframe, start=correct_date_str, end=correct_date_str).df
print(current_portfolio_df)
# Reorganize the DataFrame
# Separate ticker data
#agg_df = current_portfolio_df[current_portfolio_df['symbol']=='AGG'].drop(columns='symbol')

#qqq_df = current_portfolio_df[current_portfolio_df['symbol']=='QQQ'].drop(columns='symbol')
#spy_df = current_portfolio_df[current_portfolio_df['symbol']=='SPY'].drop(columns='symbol')
#iwm_df = current_portfolio_df[current_portfolio_df['symbol']=='IWM'].drop(columns='symbol')
# Concatenate the ticker DataFrames
#portfolio_concat_df = pd.concat([qqq_df, spy_df, iwm_df], axis=1, keys=['QQQ', 'SPY', 'IWM'])
# Preview DataFrame
#portfolio_concat_df.head()

                            close     high       low  trade_count     open  \
timestamp                                                                    
2024-05-06 04:00:00+00:00  181.71  184.200  180.4200       897162  182.354   
2024-05-06 04:00:00+00:00  188.70  188.745  184.8000       394313  186.280   
2024-05-06 04:00:00+00:00  168.10  168.140  166.0300       312762  167.460   
2024-05-06 04:00:00+00:00  204.51  204.780  203.3800       158082  203.470   
2024-05-06 04:00:00+00:00  148.58  150.050  147.9100        96375  149.870   
2024-05-06 04:00:00+00:00  192.00  192.200  189.8150       125575  191.730   
2024-05-06 04:00:00+00:00  465.68  466.160  453.3400       293163  455.580   
2024-05-06 04:00:00+00:00  413.54  413.930  406.3701       350746  408.760   
2024-05-06 04:00:00+00:00  921.40  922.200  890.5500       830423  893.900   
2024-05-06 04:00:00+00:00  440.25  440.340  436.5200       312296  437.300   
2024-05-06 04:00:00+00:00  516.57  516.610  513.3000       44442

In [25]:
if 'symbol' in current_portfolio_df.columns:
    # Separate data by ticker and drop the 'symbol' column
    spy_df = current_portfolio_df[current_portfolio_df['symbol'] == 'SPY'].drop(columns='symbol', errors='ignore')
    qqq_df = current_portfolio_df[current_portfolio_df['symbol'] == 'QQQ'].drop(columns='symbol', errors='ignore')
    iwm_df = current_portfolio_df[current_portfolio_df['symbol'] == 'IWM'].drop(columns='symbol', errors='ignore')
    aapl_df = current_portfolio_df[current_portfolio_df['symbol'] == 'AAPL'].drop(columns='symbol', errors='ignore')
    msft_df = current_portfolio_df[current_portfolio_df['symbol'] == 'MSFT'].drop(columns='symbol', errors='ignore')
    amzn_df = current_portfolio_df[current_portfolio_df['symbol'] == 'AMZN'].drop(columns='symbol', errors='ignore')
    googl_df = current_portfolio_df[current_portfolio_df['symbol'] == 'GOOGL'].drop(columns='symbol', errors='ignore')
    meta_df = current_portfolio_df[current_portfolio_df['symbol'] == 'META'].drop(columns='symbol', errors='ignore')
    tsla_df = current_portfolio_df[current_portfolio_df['symbol'] == 'TSLA'].drop(columns='symbol', errors='ignore')
    nvda_df = current_portfolio_df[current_portfolio_df['symbol'] == 'NVDA'].drop(columns='symbol', errors='ignore')
    jnj_df = current_portfolio_df[current_portfolio_df['symbol'] == 'JNJ'].drop(columns='symbol', errors='ignore')
    jpm_df = current_portfolio_df[current_portfolio_df['symbol'] == 'JPM'].drop(columns='symbol', errors='ignore')

    # Concatenate the ticker DataFrames along the columns axis
    portfolio_concat_df = pd.concat([spy_df, qqq_df, iwm_df, aapl_df, msft_df, amzn_df, googl_df, meta_df, tsla_df, nvda_df, jnj_df, jpm_df],
                                    axis=1, 
                                    keys=['SPY', 'QQQ', 'IWM', 'AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'TSLA', 'NVDA', 'JNJ', 'JPM'])
    print(portfolio_concat_df.head())
else:
    print("Symbol column not found in DataFrame.")


                              SPY                                     \
                            close    high    low trade_count    open   
timestamp                                                              
2024-05-06 04:00:00+00:00  516.57  516.61  513.3      444425  513.75   

                                                    QQQ                  ...  \
                             volume        vwap   close    high     low  ...   
timestamp                                                                ...   
2024-05-06 04:00:00+00:00  47498316  514.646463  440.25  440.34  436.52  ...   

                              JNJ                         JPM                  \
                             open   volume        vwap  close   high      low   
timestamp                                                                       
2024-05-06 04:00:00+00:00  149.87  5043249  148.607508  192.0  192.2  189.815   

                                                                 

In [42]:

# Calculate daily returns for each stock
returns_df = portfolio_concat_df.loc[:, (slice(None), 'close')].pct_change()

# Calculate daily return for the benchmark (SPY)
benchmark_return = returns_df.loc[:, ('SPY', 'close')]

# Calculate relative performance for each stock
relative_performance = returns_df.sub(benchmark_return, axis=0)

# Print the relative performance
print("Relative Performance:")
print(relative_performance)

# Calculate total volume for each stock
total_volume = portfolio_concat_df.loc[:, (slice(None), 'volume')].sum()

# Sort the stocks by total volume in descending order
volume_ranking = total_volume.sort_values(ascending=False)

# Print the volume ranking
print("Volume Ranking:")
print(volume_ranking)

# Identify the top 3 most actively traded stocks
top_traded_stocks = volume_ranking.index.get_level_values(0)[:3].tolist()
print(f"\nTop 3 Most Actively Traded Stocks: {', '.join(top_traded_stocks)}")



Relative Performance:
                            SPY   QQQ   IWM  AAPL  MSFT  AMZN GOOGL  META  \
                          close close close close close close close close   
timestamp                                                                   
2024-05-06 04:00:00+00:00   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

                           TSLA  NVDA   JNJ   JPM  
                          close close close close  
timestamp                                          
2024-05-06 04:00:00+00:00   NaN   NaN   NaN   NaN  
Volume Ranking:
TSLA   volume    84360589
AAPL   volume    79406955
SPY    volume    47498316
NVDA   volume    37463157
AMZN   volume    34805129
QQQ    volume    30214293
IWM    volume    23207978
GOOGL  volume    21866929
MSFT   volume    16990308
META   volume    15085335
JPM    volume     7910478
JNJ    volume     5043249
dtype: int64

Top 3 Most Actively Traded Stocks: TSLA, AAPL, SPY


In [6]:
# Calculate the start date (3 months ago)
start_date = now - timedelta(days=90)

# Convert to pandas Timestamp with timezone and format as ISO 8601 string
start_timestamp = pd.Timestamp(start_date).normalize().isoformat()
end_timestamp = pd.Timestamp(now).normalize().isoformat()

# Set the tickers
tickers = ["QQQ", "SPY", "IWM"]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get historical closing prices for the tickers
historical_data_df = api.get_bars(tickers, timeframe, start=start_timestamp, end=end_timestamp).df

# Reorganize the DataFrame
historical_data_df = historical_data_df.reset_index()
historical_data_df['timestamp'] = pd.to_datetime(historical_data_df['timestamp'])

# Pivot the DataFrame to have tickers as columns
pivoted_df = historical_data_df.pivot(index='timestamp', columns='symbol', values='close')

# Normalize the prices by dividing by the first price for each ticker
normalized_df = pivoted_df.div(pivoted_df.iloc[0])

# Create a line plot using hvplot
line_plot = normalized_df.hvplot.line(xlabel='Date', ylabel='Relative Price', width=800, height=400, legend='top')

# Display the plot
line_plot

:NdOverlay   [symbol]
   :Curve   [timestamp]   (value)

In [19]:
# Define the directory path for raw_data
raw_data_dir = '../data/raw_data'
if not os.path.exists(raw_data_dir):
    os.makedirs(raw_data_dir)

# Function to save DataFrame to CSV or log if empty
def save_data(df, filename, description):
    if not df.empty:
        df.to_csv(filename)
        print(f"{description} data saved successfully to {filename}")
    else:
        print(f"No {description} data to save. Writing null placeholder.")
        # Write a null placeholder to indicate no data was fetched
        with open(filename, 'w') as file:
            file.write('null')

try:
    # Check if the DataFrame exists and isn't empty
    if 'portfolio_concat_df' in locals() and not portfolio_concat_df.empty:
        stock_file_path = os.path.join(raw_data_dir, 'stock_data.csv')
        save_data(portfolio_concat_df, stock_file_path, 'Stock')
    else:
        save_data(pd.DataFrame(), os.path.join(raw_data_dir, 'stock_data.csv'), 'Stock')

    if 'crypto_concat_df' in locals() and not crypto_concat_df.empty:
        crypto_file_path = os.path.join(raw_data_dir, 'crypto_data.csv')
        save_data(crypto_concat_df, crypto_file_path, 'Crypto')
    else:
        save_data(pd.DataFrame(), os.path.join(raw_data_dir, 'crypto_data.csv'), 'Crypto')
except Exception as e:
    print(f"An error occurred while saving data: {str(e)}")


:Overlay
   .Segments.I   :Segments   [timestamp,low,timestamp,high]   (close,high,low,trade_count,lbound,ubound)
   .Rectangles.I :Rectangles   [lbound,close,ubound,trade_count]   (close,high,low,trade_count,lbound,ubound)